In [2]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'

# %%


from datetime import date
import pandas as pd
import time
import sys
import traceback
# local modules
from binance.client import Client
from binance.client import BaseClient
from binance.enums import *
from indicator import indicators


# local file
import secrets
import json
import yfinance as yf
import numpy as np
import datetime as dt
import requests 
import json 
import pandas as pd 
import numpy as np  
import requests
import time
import urllib
from finta import TA

# %%

df3 = pd.read_csv("parameters.csv")
df3.set_index("parameters",inplace=True)
df2=df3.T

import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(formatter)

file_handler = logging.FileHandler('logs.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(stdout_handler)
BaseClient.FUTURES_TESTNET_URL='https://fapi.binance.{}/fapi'
BaseClient.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'

# client = Client("cc9feab03d1264ed67c07738cdd42502dd80a8b67fedaf2e5f9b6e9c55a2faad", "e30d3db72358639f29b6280bf1c54fd564e7b1eb5cb13f020739fd197f396e1b")  #elclis
client = Client("86c637e3177c4280ca59adfeaab720357561be2c817eccf84cc0e3470a666c93", "f77d6c868d6caf55cb9b04e980b7aa18ce729fc1685ab1417e1bea820c7ac3ef")  #elclis


In [3]:
order = client.futures_create_order(
    symbol = "BTCUSDT",
    side = SIDE_SELL,
    positionSide = "LONG",
    type = ORDER_TYPE_MARKET,
    quantity = 1,
    )

In [ ]:
pd.options.mode.chained_assignment = None



def StochRSI(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() /          downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()

    return stochrsi, stochrsi_K, stochrsi_D

# %%
def candle(symbol, interval):

    global df2,client

    client = Client("GBCTCkf6qgDQSZrPJWp513J69pJ2yVC8Fntdos7REMs5kyWn4ICJ2FNKnX9CM7WW", "v0gKOvAfruQaXGbk77W1CsIWf9CVR9kL0U2DEyru2pUwAapXrfyfAMGrEZIdSyaN")  #sudhanshu real api

    BaseClient.FUTURES_TESTNET_URL='https://fapi.binance.{}/fapi'
    BaseClient.FUTURES_URL = 'https://fapi.binance.{}/fapi'
    BaseClient.testnet=False
    data=client.futures_klines(symbol=symbol,interval=interval)

    # BaseClient.FUTURES_TESTNET_URL='https://fapi.binance.{}/fapi'
    # BaseClient.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'
    client = Client("cc9feab03d1264ed67c07738cdd42502dd80a8b67fedaf2e5f9b6e9c55a2faad", "e30d3db72358639f29b6280bf1c54fd564e7b1eb5cb13f020739fd197f396e1b")  #sudhanshu real api


    df = pd.DataFrame(data)
    df.columns = ['Datetime',
                'Open', 'High', 'Low', 'Close', 'volume',
                'close_time', 'qav', 'num_trades',
                'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.close_time]
    
    df.drop(['close_time','qav','num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'],axis=1,inplace=True)
           
    
    
    df['Open']=pd.to_numeric(df["Open"], downcast="float")
    df["High"]=pd.to_numeric(df["High"], downcast="float")
    df["Low"]=pd.to_numeric(df["Low"], downcast="float")
    df["Close"]=pd.to_numeric(df["Close"], downcast="float")
    df["volume"]=pd.to_numeric(df["volume"], downcast="float")
    df['ATR']=TA.ATR(df,int(df2['ATR'][0]))
    print(df)
    HAdf = df[['Open', 'High', 'Low', 'Close']]

    HAdf['Close'] = round(((df['Open'] + df['High'] + df['Low'] + df['Close'])/4),4)


    for i in range(len(df)):
        if i == 0:
            HAdf.iat[0,0] = round(((df['Open'].iloc[0] + df['Close'].iloc[0])/2),4)
        else:
            HAdf.iat[i,0] = round(((HAdf.iat[i-1,0] + HAdf.iat[i-1,3])/2),4)

    HAdf['High'] = HAdf.loc[:,['Open', 'Close']].join(df['High']).max(axis=1)
    HAdf['Low'] = HAdf.loc[:,['Open', 'Close']].join(df['Low']).min(axis=1)


    HAdf['Stoch-k']=StochRSI(HAdf['Close'],int(df2['STOCH-PERIOD'][0]),int(df2['STOCH-K'][0]),int(df2['STOCH-D'][0]))[1]*100
    HAdf['Stoch-d']=StochRSI(HAdf['Close'],int(df2['STOCH-PERIOD'][0]),int(df2['STOCH-K'][0]),int(df2['STOCH-D'][0]))[2]*100
    HAdf['RSI']=TA.RSI(HAdf,int(df2['RSI'][0]))
    HAdf['ATR']=df['ATR']
    # df1['STOCH-k']=TA.STOCH(df1,14)
    # df1['STOCH-D']=TA.STOCHD(df1,14)

    return HAdf[:-1]



In [ ]:
candle('BTCUSDT','5m')

In [ ]:
order = client.futures_create_order(
    symbol = "BTCUSDT",
    side = SIDE_BUY,
    positionSide = "LONG",
    type = ORDER_TYPE_MARKET,
    quantity = 1,
    )

In [ ]:
client.futures_symbol_ticker(symbol='BTCUSDT')['price']

In [ ]:
import pandas as pd
df3 = pd.read_csv("parameters.csv")
df3.set_index("parameters",inplace=True)
df2=df3.T

df2['step'][0]

In [3]:
df2

parameters,binance_api_key,binance_api_secret_key,take_profit_precentage,stop_loss_precentage,ATR,STOCH-PERIOD,STOCH-K,STOCH-D,RSI,investment,symbol,ATR MULTIPLY,binance_X,time_frame,time_in_mins,difference,bot
value,Ho3aQlXSESZrQ5LYuu5FqIctzqxJ5EcDDB6FFLZwICHem3...,BI1XvhBgoN2eUfcWJLz4tXWVuSTIw0WlTFCJkajqhT6cFY...,2,10,5,14,3,3,14,100,BTCUSDT,1.1,5,1m,1,0.1,on
Unnamed: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETHUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BNBUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADAUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SOLUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AVAXUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XRPUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOGEUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRVUSDT,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FTMUSDT,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import math
import pandas as pd

from binance.client import Client
import json
client1 = Client("GBCTCkf6qgDQSZrPJWp513J69pJ2yVC8Fntdos7REMs5kyWn4ICJ2FNKnX9CM7WW", "v0gKOvAfruQaXGbk77W1CsIWf9CVR9kL0U2DEyru2pUwAapXrfyfAMGrEZIdSyaN")  #sudhanshu real api

exchangeName = "binance"




class Round:
    def step_size_to_precision(self, ss):
        return ss.find('1') - 1

    def format_value(self, val, step_size_str):
        precision = self.step_size_to_precision(step_size_str)
        if precision > 0:
            return "{:0.0{}f}".format(val, precision)
        return math.floor(int(val))

    def format_valueDown(self, val, step_size_str):
        precision = self.step_size_to_precision(step_size_str)
        if precision > 0:
            return "{:0.0{}f}".format(val, precision)
        return math.trunc(int(val))


df = pd.DataFrame(TokenDetails["symbols"])

for index, ticker in df.iterrows():
    for tickerRounding in ticker["filters"]:
        try:
            if tickerRounding["filterType"] == "LOT_SIZE":
                print(tickerRounding,ticker)


        except:
            pass

In [ ]:
info = client.futures_exchange_info()
def get_precision(symbol):
    global info
    for x in info['symbols']:
        if x['symbol'] == symbol:
            return x['quantityPrecision'],x['pricePrecision']

In [ ]:
get_precision('BTCUSDT')[0]

In [ ]:
root_url = 'https://api.binance.com/fapi/v1/klines'
url = root_url + '?symbol=' + 'BTCUSDT' + '&interval=' + '5m'
url